# Titan’s AI multimodal embeddings for better e-commerce recommendations



In [1]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')

In [2]:
image_path = "data/bags.png"


In [3]:
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [4]:

aws_region_name = "us-east-1"

try:
    
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region_name
    )
except NoCredentialsError:
    print("Credentials not found. Please configure your AWS profile.")

In [5]:
image_paths = ["1.png", "2.png", "3.png", "4.png", "5.png", "6.png", "7.png"]
descriptions = ["Red bag", "Brown bag", "Blue bag", "Pink bag", "Mint bag", "Salmon bag", "Black bag"]

In [6]:
import boto3
from botocore.exceptions import NoCredentialsError
import base64
from IPython.core.display import HTML
import json
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import uuid

In [7]:
# Generate a unique UUID for each product
ids = [str(uuid.uuid4()) for _ in range(len(image_paths))]

# Create a DataFrame with columns for ID, image, and vector
df = pd.DataFrame({'ID': ids, 'image': image_paths, 'description': descriptions})
df

,ID,image,description
0,96ee7533-e057-4d01-bd2a-22bde5372120,1.png,Red bag
1,4948a209-adad-4cc2-8ca1-7be2c34544c2,2.png,Brown bag
2,090f4603-d993-4347-8593-40ecfccb0929,3.png,Blue bag
3,650d14bd-fefa-4d12-9dc1-31439b89d4bc,4.png,Pink bag
4,268a32e9-79c2-495a-8f59-c39a26ba9766,5.png,Mint bag
5,209971d5-de92-4ed8-b143-ff2ffa099859,6.png,Salmon bag
6,921ef916-5e05-413c-ac90-4c2e109bbb32,7.png,Black bag


In [8]:
def generate_embeddings(image_base64=None, text_description=None):
    input_data = {}

    if image_base64 is not None:
        input_data["inputImage"] = image_base64
    if text_description is not None:
        input_data["inputText"] = text_description

    if not input_data:
        raise ValueError("At least one of image_base64 or text_description must be provided")

    body = json.dumps(input_data)

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId="amazon.titan-embed-image-v1",
        accept="application/json",
        contentType="application/json"
    )

    response_body = json.loads(response.get("body").read())

    finish_reason = response_body.get("message")

    if finish_reason is not None:
        raise EmbedError(f"Embeddings generation error: {finish_reason}")

    return response_body.get("embedding")


In [9]:
def base64_encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf8')

In [11]:
embeddings = []
for index, row in df.iterrows():
    #full_image_path = os.path.join(image_path, row['image'])
    image_base64 = base64_encode_image(row['image'])
    embedding = generate_embeddings(image_base64=image_base64)
    embeddings.append(embedding)

In [12]:
df['image_embedding'] = embeddings
df

,ID,image,description,image_embedding
0,96ee7533-e057-4d01-bd2a-22bde5372120,1.png,Red bag,"[0.010198408, 0.023406181, -0.04982173, -0.024..."
1,4948a209-adad-4cc2-8ca1-7be2c34544c2,2.png,Brown bag,"[0.03369583, 0.0038290713, -0.006622041, -0.02..."
2,090f4603-d993-4347-8593-40ecfccb0929,3.png,Blue bag,"[0.022023838, 0.032252483, 0.012624543, -0.023..."
3,650d14bd-fefa-4d12-9dc1-31439b89d4bc,4.png,Pink bag,"[-0.0019618955, 0.018727183, -0.0022740152, -0..."
4,268a32e9-79c2-495a-8f59-c39a26ba9766,5.png,Mint bag,"[0.03183198, 0.017325217, 0.014175178, -0.0139..."
5,209971d5-de92-4ed8-b143-ff2ffa099859,6.png,Salmon bag,"[-0.003023843, 0.009821806, -0.007275412, -0.0..."
6,921ef916-5e05-413c-ac90-4c2e109bbb32,7.png,Black bag,"[0.016667765, 0.0008072085, -0.0142243225, -0...."


In [22]:
customer_query = "Hi! I'm looking for a pink bag"
query_embedding = generate_embeddings(text_description=customer_query)
query_embedding

[0.0087890625,
 0.016479492,
 -0.048339844,
 -0.0071105957,
 -0.0013122559,
 0.035888672,
 0.080566406,
 -0.013000488,
 -0.0055236816,
 -0.022705078,
 0.011962891,
 -0.009216309,
 0.034179688,
 -0.02722168,
 0.041748047,
 0.0025482178,
 -0.010437012,
 -0.020019531,
 0.019897461,
 -0.016723633,
 -0.006500244,
 0.044433594,
 -0.037841797,
 0.03149414,
 0.0074157715,
 -0.031982422,
 0.05444336,
 -0.03466797,
 -0.030273438,
 -0.04638672,
 0.041015625,
 -0.034179688,
 0.03857422,
 0.03466797,
 -0.0014953613,
 -0.026123047,
 0.00680542,
 0.03857422,
 0.007232666,
 0.036865234,
 0.041015625,
 -0.05078125,
 -0.03125,
 0.032226562,
 -0.039794922,
 0.025146484,
 0.015197754,
 -0.008300781,
 -0.029541016,
 0.037597656,
 0.015258789,
 0.03564453,
 0.041503906,
 -0.012390137,
 2.670288e-05,
 0.033935547,
 0.0073242188,
 0.00079345703,
 -0.057861328,
 -0.00076293945,
 -0.026611328,
 -0.025024414,
 0.068847656,
 -0.022949219,
 -0.018188477,
 0.03857422,
 0.037597656,
 0.014038086,
 -0.012145996,
 -0.

In [23]:
vectors = list(df['image_embedding'])


In [24]:
cosine_scores = cosine_similarity([query_embedding], vectors)[0]


In [25]:
score_series = pd.Series(cosine_scores, index = df['image'])


In [26]:
sorted_scores = score_series.sort_values(ascending=False)
sorted_scores

image
3.png    0.448238
6.png    0.416365
1.png    0.400541
5.png    0.380403
7.png    0.370764
2.png    0.368642
4.png    0.357355
dtype: float64

In [27]:
from IPython.core.display import display, HTML

/tmp/ipykernel_16780/2058709175.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [28]:
html_str = f"<h3>Query: '{customer_query}'</h3><table><tr>"


In [29]:
# Loop through sorted scores and images
for filename, score in sorted_scores.items():
    #image_path = os.path.join('data', 'bags', filename)
    image_path = os.path.join( filename)
    print(image_path)
    # Adding each image and its details to the HTML string
    html_str += f"<td style='text-align:center'><img src='{image_path}' width='100'><br>{filename}<br>Score: {score:.3f}</td>"

3.png
6.png
1.png
5.png
7.png
2.png
4.png


In [30]:


html_str += "</tr></table>"

# Display the HTML
display(HTML(html_str))

3.pngScore: 0.448,6.pngScore: 0.416,1.pngScore: 0.401,5.pngScore: 0.380,7.pngScore: 0.371,2.pngScore: 0.369,4.pngScore: 0.357
